In [2]:
# In this version of OGA with random dictionaries, we use QMC to evaluate the loss function. 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 
pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_dtype(torch.float64)

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1


def show_convergence_order(err_l2,exponent,dict_size, filename,write2file = False):
    
    if write2file:
        file_mode = "a" if os.path.exists(filename) else "w"
        f_write = open(filename, file_mode)
    
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    if write2file:
        f_write.write('dictionary size: {}\n'.format(dict_size))
        f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t {:.6f} \t\t *  \n".format(neuron_nums[i],item ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t * \t\t \n".format(neuron_nums[i],item ))
        else: 
            print("{} \t\t {:.6f} \t\t {:.6f} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )
            if write2file: 
                f_write.write("{} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ))
    if write2file:     
        f_write.write("\n")
        f_write.close()

def show_convergence_order_latex(err_l2,exponent,k=1,d=1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    l2_order = -1/2-(2*k + 1)/(2*d)
    print("neuron num  & \t $\\|u-u_n \\|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$  \\\\ \\hline \\hline ".format(l2_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t & {:.6f} &\t\t *  \\\ \hline  \n".format(neuron_nums[i],item) )   
        else: 
            print("{} \t\t &  {:.3e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2) ) )


def generate_relu_dict4plusD_QMC(dim, s,N0):

    samples = torch.rand(s*N0,dim)  

    # for i in range(s-1):
        # samples = torch.cat([samples,Sob.draw(N0).double()],0)
    # Form the transformation matrix and shift vector 
    diagonal = torch.ones(dim)*pi  
    diagonal[-1] =  2*dim**0.5
    diagonal[-2] = 2*pi 
    T = torch.diag(diagonal)

    shift = torch.zeros(dim)
    shift[-1] = -dim**0.5 
    samples = samples@T + shift 

    Wb_tensor = torch.ones(s*N0,dim+1) # each neuron parameter stored in rows  
    for i in range(dim): # 0, 1, ... dim-1 
        for j in range(i+1): # 0, 1, ... i 
            if i == 0: 
                Wb_tensor[:,i] = Wb_tensor[:,i]*torch.cos(samples[:,j])
            if i == (dim - 1):
                if j != i:
                    Wb_tensor[:,i] = Wb_tensor[:,i] * torch.sin(samples[:,j]) 
            if i != 0 and i != (dim - 1): 
                if j != i: 
                    Wb_tensor[:,i] = Wb_tensor[:,i] * torch.sin(samples[:,j]) 
                else: 
                    Wb_tensor[:,i] = Wb_tensor[:,i] * torch.cos(samples[:,j]) 
            
    Wb_tensor[:,dim] = samples[:,-1] 

    return Wb_tensor.to(device)


def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights.to(device), integration_points.to(device) 


def minimize_linear_layer_explicit_assemble(model,target,weights, integration_points,solver="direct",memory=2**29):
    """
    """
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    
    # new batched operation 
    n = b.size(0)
    M = integration_points.size(0)
    
    total_size = n * M # memory, number of floating numbers 
    num_batch = total_size//memory + 1 # divide according to memory
    batch_size = M//num_batch
    start_ind = 0
    end_ind = 0 
    jac = torch.zeros(b.size(0),b.size(0)).to(device)
    rhs = torch.zeros(b.size(0),1).to(device)
#     print("mat assemble, number batches: ",num_batch)
    for j in range(0,M,batch_size): 
        end_ind = j + batch_size
        basis_value_col = F.relu(integration_points[j:end_ind] @ w.t()+ b)**(model.k) 
        weighted_basis_value_col = basis_value_col * weights[j:end_ind] 
        jac += weighted_basis_value_col.t() @ basis_value_col 
        rhs += weighted_basis_value_col.t() @ (target(integration_points[j:end_ind,:])) 
        
    print("jac: ", jac.device)
    print("assembling the matrix time taken: ", time.time()-start_time) 
    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 

def OGAL2FittingReLU4Dplus_QMC(my_model,target,s,N0,num_epochs, M, k =1, linear_solver = "direct", memory = 2**29): 
    
    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    # samples for QMC integral
    dim = 10 
    start_time = time.time()
    # Sob_integral = torch.quasirandom.SobolEngine(dimension =4, scramble= False, seed=None) 
    # integration_points = Sob_integral.draw(M).double() 
    # integration_points = integration_points.to(device)
    weights, integration_points = MonteCarlo_Sobol_dDim_weights_points(M ,d = dim) 
    _, integration_points_test = MonteCarlo_Sobol_dDim_weights_points(M*2 ,d = dim) 
    print("generate sob sequence:", time.time() - start_time) 

    err = torch.zeros(num_epochs+1)
    if my_model == None: 
        func_values = target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []

    else: 
        func_values = target(integration_points) - my_model(integration_points).detach()
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done

    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size() ) 

    err[0]= torch.mean(func_values_sqrd)**0.5
    all_start_time = time.time()
    
    solver = linear_solver
    print("using linear solver: ",solver)
    for i in range(num_epochs): 
        relu_dict_parameters = generate_relu_dict4plusD_QMC(dim, s,N0).t()   
        print("epoch: ",i+1, end = '\t')
        if num_neuron == 0: 
            func_values = target(integration_points)
        else: 
            

            total_size = num_neuron * M 
            num_batch = total_size//(2**30) + 1 # divide according to memory
            batch_size = M//num_batch
            end_ind = 0 

            func_values = torch.zeros(M,1).to(device)

            for j in range(0,M,batch_size): 
                end_ind = j + batch_size
                func_values[j:end_ind,:] = target(integration_points[j:end_ind,:]) - my_model(integration_points[j:end_ind,:]).detach()

        start_time = time.time() 
        
        M = integration_points.size(0)
        N = s*N0 
        output = torch.zeros(N,1)
        num_batches = (N*M)//memory + 1 # decide num_batches according to memory 
        batch_size = N//num_batches 
        print("argmax batch num, ", num_batches)
        for j in range(0,N,batch_size): 

            end_index = j + batch_size  
            basis_values_batch = (F.relu( torch.matmul(integration_points,relu_dict_parameters[0:dim, j:end_index] ) - relu_dict_parameters[dim, j:end_index])**k).T # uses broadcasting    
            output[j:end_index,0]  = (torch.abs(torch.matmul(basis_values_batch,func_values))/M)[:,0]
            
        neuron_index = torch.argmax(output.flatten())
        
#         basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T # uses broadcasting
#         output = torch.abs(torch.matmul(basis_values,func_values))/M # 
#         neuron_index = torch.argmax(output.flatten())
        print("argmax time taken, ", time.time() - start_time)
        
        list_w.append(relu_dict_parameters[0:dim, neuron_index]) # 
        list_b.append(-relu_dict_parameters[dim,neuron_index])
        num_neuron += 1
        my_model = model(dim,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        start_time = time.time() 
        sol = minimize_linear_layer_explicit_assemble(my_model,target,weights,integration_points, solver,memory)
        print("\t\t time taken minimize linear layer: ",time.time() - start_time) 
        my_model.fc2.weight.data[0,:] = sol[:]

        # calculate the test error
#         func_values = target(integration_points_test) - my_model(integration_points_test).detach()
        
        M2 = integration_points_test.size(0)
        total_size = num_neuron * M2 
        num_batch = total_size//memory + 1 # divide according to memory
        batch_size = M//num_batch
        end_ind = 0 

        func_values_sqrd = torch.zeros(M2,1)  
        
        for j in range(0,M2,batch_size): 
            end_ind = j + batch_size
            func_values_sqrd[j:end_ind,:] = target(integration_points_test[j:end_ind,:]) - my_model(integration_points_test[j:end_ind,:]).detach()
        
        func_values_sqrd = func_values_sqrd**2 

        #Todo Done 
        err[i+1]= torch.mean(func_values_sqrd)**0.5
        print("current error: ",err[i+1]) 
    print("total duration: ",time.time() - all_start_time)
    return err, my_model

def print_convergence_order(err, neuron_num_exponent): 

    neuron_nums = [2**j for j in range(2,neuron_num_exponent)]
    err_list = [err[i] for i in neuron_nums ] 

    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t")
            print("*")
        else: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t") 
            print(np.log(err_list[i-1]/err_list[i])/np.log(2))



In [5]:
def target(x): ## Gaussian function in dimension 10  
    d = 10 
    cn =   7.03/d 
    return torch.exp(-torch.sum( cn**2 * (x - 0.5)**2,dim = 1, keepdim = True)) 

dim = 10 
function_name = "10DGaussian"
filename_write = "data/1DOGA-{}-order.txt".format(function_name)
M = 2**20 # around2**19 50w 
f_write = open(filename_write, "a")
f_write.write("Integration points: Quasi Monte Carlo:  {}\n".format(M))
f_write.close() 
save = True 
write2file = True
memory = 2**30 
for relu_k in [1]: 
    s = 1 
    for N0 in [2**10]: 
        print()
        print() 
        N = s*N0 
        exponent = 9    
        num_epochs=  2**exponent 
        my_model = None 

        err, my_model = OGAL2FittingReLU4Dplus_QMC(my_model,target, \
                    s,N0,num_epochs, M, k = relu_k, linear_solver = "direct", memory=memory)

        if save: 
            folder = 'data/'
            filename = folder + function_name + "_err_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(err,filename)
            filename = folder + function_name +  "_model_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(my_model.state_dict(),filename) 

        show_convergence_order(err,exponent,N,filename_write,write2file = write2file)
        show_convergence_order_latex(err,exponent,k=relu_k,d=dim)





generate sob sequence: 0.09398794174194336
using linear solver:  direct
epoch:  1	argmax batch num,  2
argmax time taken,  0.04302263259887695
jac:  cuda:0
assembling the matrix time taken:  0.00034689903259277344
solving Ax = b time taken:  0.0004584789276123047
		 time taken minimize linear layer:  0.000827789306640625
current error:  tensor(0.0924)
epoch:  2	argmax batch num,  2
argmax time taken,  0.04302549362182617
jac:  cuda:0
assembling the matrix time taken:  0.00039076805114746094
solving Ax = b time taken:  0.0008771419525146484
		 time taken minimize linear layer:  0.0012917518615722656
current error:  tensor(0.0788)
epoch:  3	argmax batch num,  2
argmax time taken,  0.04333353042602539
jac:  cuda:0
assembling the matrix time taken:  0.0003581047058105469
solving Ax = b time taken:  0.0009264945983886719
		 time taken minimize linear layer:  0.0013089179992675781
current error:  tensor(0.0771)
epoch:  4	argmax batch num,  2
argmax time taken,  0.0433652400970459
jac:  cud

argmax time taken,  0.042664527893066406
jac:  cuda:0
assembling the matrix time taken:  0.0003407001495361328
solving Ax = b time taken:  0.002462148666381836
		 time taken minimize linear layer:  0.0028264522552490234
current error:  tensor(0.0467)
epoch:  32	argmax batch num,  2
argmax time taken,  0.04280591011047363
jac:  cuda:0
assembling the matrix time taken:  0.0003688335418701172
solving Ax = b time taken:  0.002474069595336914
		 time taken minimize linear layer:  0.0028662681579589844
current error:  tensor(0.0454)
epoch:  33	argmax batch num,  2
argmax time taken,  0.04267597198486328
jac:  cuda:0
assembling the matrix time taken:  0.00035190582275390625
solving Ax = b time taken:  0.0027806758880615234
		 time taken minimize linear layer:  0.0031576156616210938
current error:  tensor(0.0451)
epoch:  34	argmax batch num,  2
argmax time taken,  0.045980215072631836
jac:  cuda:0
assembling the matrix time taken:  0.000362396240234375
solving Ax = b time taken:  0.00273251533

current error:  tensor(0.0359)
epoch:  61	argmax batch num,  2
argmax time taken,  0.046808719635009766
jac:  cuda:0
assembling the matrix time taken:  0.0003383159637451172
solving Ax = b time taken:  0.004320621490478516
		 time taken minimize linear layer:  0.004683732986450195
current error:  tensor(0.0357)
epoch:  62	argmax batch num,  2
argmax time taken,  0.04677724838256836
jac:  cuda:0
assembling the matrix time taken:  0.00035762786865234375
solving Ax = b time taken:  0.00436854362487793
		 time taken minimize linear layer:  0.004750728607177734
current error:  tensor(0.0355)
epoch:  63	argmax batch num,  2
argmax time taken,  0.04699087142944336
jac:  cuda:0
assembling the matrix time taken:  0.00034165382385253906
solving Ax = b time taken:  0.004423379898071289
		 time taken minimize linear layer:  0.004789113998413086
current error:  tensor(0.0352)
epoch:  64	argmax batch num,  2
argmax time taken,  0.04711103439331055
jac:  cuda:0
assembling the matrix time taken:  0.00

argmax time taken,  0.047635555267333984
jac:  cuda:0
assembling the matrix time taken:  0.00037932395935058594
solving Ax = b time taken:  0.007811307907104492
		 time taken minimize linear layer:  0.008215904235839844
current error:  tensor(0.0274)
epoch:  91	argmax batch num,  2
argmax time taken,  0.04758095741271973
jac:  cuda:0
assembling the matrix time taken:  0.0003535747528076172
solving Ax = b time taken:  0.007546663284301758
		 time taken minimize linear layer:  0.007925987243652344
current error:  tensor(0.0273)
epoch:  92	argmax batch num,  2
argmax time taken,  0.04771018028259277
jac:  cuda:0
assembling the matrix time taken:  0.00038313865661621094
solving Ax = b time taken:  0.007493495941162109
		 time taken minimize linear layer:  0.007901906967163086
current error:  tensor(0.0270)
epoch:  93	argmax batch num,  2
argmax time taken,  0.047563791275024414
jac:  cuda:0
assembling the matrix time taken:  0.00035572052001953125
solving Ax = b time taken:  0.007627725601

argmax time taken,  0.04840970039367676
jac:  cuda:0
assembling the matrix time taken:  0.00040340423583984375
solving Ax = b time taken:  0.009510278701782227
		 time taken minimize linear layer:  0.009939432144165039
current error:  tensor(0.0219)
epoch:  123	argmax batch num,  2
argmax time taken,  0.04847550392150879
jac:  cuda:0
assembling the matrix time taken:  0.0003695487976074219
solving Ax = b time taken:  0.009164571762084961
		 time taken minimize linear layer:  0.009558916091918945
current error:  tensor(0.0219)
epoch:  124	argmax batch num,  2
argmax time taken,  0.048452138900756836
jac:  cuda:0
assembling the matrix time taken:  0.00040793418884277344
solving Ax = b time taken:  0.009217977523803711
		 time taken minimize linear layer:  0.00965118408203125
current error:  tensor(0.0218)
epoch:  125	argmax batch num,  2
argmax time taken,  0.0484926700592041
jac:  cuda:0
assembling the matrix time taken:  0.0003814697265625
solving Ax = b time taken:  0.0092315673828125

...

argmax time taken,  0.05956554412841797
jac:  cuda:0
assembling the matrix time taken:  0.0005197525024414062
solving Ax = b time taken:  0.04415178298950195
		 time taken minimize linear layer:  0.04469728469848633
current error:  tensor(0.0094)
epoch:  394	argmax batch num,  2
argmax time taken,  0.05914664268493652
jac:  cuda:0
assembling the matrix time taken:  0.0005159378051757812
solving Ax = b time taken:  0.046506643295288086
		 time taken minimize linear layer:  0.047185659408569336
current error:  tensor(0.0092)
epoch:  395	argmax batch num,  2
argmax time taken,  0.060147762298583984
jac:  cuda:0
assembling the matrix time taken:  0.0005095005035400391
solving Ax = b time taken:  0.04424595832824707
		 time taken minimize linear layer:  0.044780731201171875
current error:  tensor(0.0092)
epoch:  396	argmax batch num,  2
argmax time taken,  0.05918312072753906
jac:  cuda:0
assembling the matrix time taken:  0.0005333423614501953
solving Ax = b time taken:  0.0465564727783203

solving Ax = b time taken:  0.04558730125427246
		 time taken minimize linear layer:  0.04626584053039551
current error:  tensor(0.0088)
epoch:  424	argmax batch num,  2
argmax time taken,  0.05990290641784668
jac:  cuda:0
assembling the matrix time taken:  0.0005197525024414062
solving Ax = b time taken:  0.0455629825592041
		 time taken minimize linear layer:  0.04610753059387207
current error:  tensor(0.0087)
epoch:  425	argmax batch num,  2
argmax time taken,  0.05983853340148926
jac:  cuda:0
assembling the matrix time taken:  0.0005228519439697266
solving Ax = b time taken:  0.04621291160583496
		 time taken minimize linear layer:  0.046759843826293945
current error:  tensor(0.0087)
epoch:  426	argmax batch num,  2
argmax time taken,  0.05996823310852051
jac:  cuda:0
assembling the matrix time taken:  0.0005295276641845703
solving Ax = b time taken:  0.04991269111633301
		 time taken minimize linear layer:  0.050605058670043945
current error:  tensor(0.0087)
epoch:  427	argmax bat

solving Ax = b time taken:  0.05436444282531738
		 time taken minimize linear layer:  0.055077552795410156
current error:  tensor(0.0084)
epoch:  454	argmax batch num,  2
argmax time taken,  0.06092977523803711
jac:  cuda:0
assembling the matrix time taken:  0.0005631446838378906
solving Ax = b time taken:  0.05434250831604004
		 time taken minimize linear layer:  0.0549318790435791
current error:  tensor(0.0084)
epoch:  455	argmax batch num,  2
argmax time taken,  0.06103968620300293
jac:  cuda:0
assembling the matrix time taken:  0.0005316734313964844
solving Ax = b time taken:  0.05439400672912598
		 time taken minimize linear layer:  0.05495119094848633
current error:  tensor(0.0083)
epoch:  456	argmax batch num,  2
argmax time taken,  0.060951948165893555
jac:  cuda:0
assembling the matrix time taken:  0.0005629062652587891
solving Ax = b time taken:  0.0525209903717041
		 time taken minimize linear layer:  0.05311012268066406
current error:  tensor(0.0083)
epoch:  457	argmax batc

current error:  tensor(0.0080)
epoch:  483	argmax batch num,  2
argmax time taken,  0.06149148941040039
jac:  cuda:0
assembling the matrix time taken:  0.0005524158477783203
solving Ax = b time taken:  0.05582833290100098
		 time taken minimize linear layer:  0.056406259536743164
current error:  tensor(0.0080)
epoch:  484	argmax batch num,  2
argmax time taken,  0.0616457462310791
jac:  cuda:0
assembling the matrix time taken:  0.0005664825439453125
solving Ax = b time taken:  0.05591011047363281
		 time taken minimize linear layer:  0.056502342224121094
current error:  tensor(0.0080)
epoch:  485	argmax batch num,  2
argmax time taken,  0.061699867248535156
jac:  cuda:0
assembling the matrix time taken:  0.0006415843963623047
solving Ax = b time taken:  0.05607867240905762
		 time taken minimize linear layer:  0.056876420974731445
current error:  tensor(0.0080)
epoch:  486	argmax batch num,  2
argmax time taken,  0.06182360649108887
jac:  cuda:0
assembling the matrix time taken:  0.000

argmax time taken,  0.06263303756713867
jac:  cuda:0
assembling the matrix time taken:  0.0005910396575927734
solving Ax = b time taken:  0.05768442153930664
		 time taken minimize linear layer:  0.05830240249633789
current error:  tensor(0.0076)
total duration:  57.695353507995605
neuron num 		 error 		 order
4 		 0.075214 		 *  

8 		 0.068855 		 0.127440 

16 		 0.058727 		 0.229549 

32 		 0.045383 		 0.371858 

64 		 0.035039 		 0.373199 

128 		 0.021475 		 0.706328 

256 		 0.012981 		 0.726265 

512 		 0.007626 		 0.767278 

neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-0.65})$  \\ \hline \hline 
4 		 & 0.075214 &		 *  \\ \hline  

8 		 &  6.886e-02 &  		 0.13 \\ \hline  

16 		 &  5.873e-02 &  		 0.23 \\ \hline  

32 		 &  4.538e-02 &  		 0.37 \\ \hline  

64 		 &  3.504e-02 &  		 0.37 \\ \hline  

128 		 &  2.147e-02 &  		 0.71 \\ \hline  

256 		 &  1.298e-02 &  		 0.73 \\ \hline  

512 		 &  7.626e-03 &  		 0.77 \\ \hline  



In [8]:
def target(x): ## Gaussian function in dimension 10  
    d = 10 
    cn =   7.03/d 
    return torch.exp(-torch.sum( cn**2 * (x - 0.5)**2,dim = 1, keepdim = True)) 

dim = 10 
function_name = "10DGaussian"
filename_write = "data/1DOGA-{}-order.txt".format(function_name)
M = 2**20 # around2**19 50w 
f_write = open(filename_write, "a")
f_write.write("Integration points: Quasi Monte Carlo:  {}\n".format(M))
f_write.close() 
save = True 
write2file = True
memory = 2**30 
for relu_k in [4]: 
    s = 1 
    for N0 in [2**11]: 
        print()
        print() 
        N = s*N0 
        exponent = 9    
        num_epochs=  2**exponent 
        my_model = None 

        err, my_model = OGAL2FittingReLU4Dplus_QMC(my_model,target, \
                    s,N0,num_epochs, M, k = relu_k, linear_solver = "direct", memory=memory)

        if save: 
            folder = 'data/'
            filename = folder + function_name + "_err_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(err,filename)
            filename = folder + function_name +  "_model_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(my_model.state_dict(),filename) 

        show_convergence_order(err,exponent,N,filename_write,write2file = write2file)
        show_convergence_order_latex(err,exponent,k=relu_k,d=dim)





generate sob sequence: 0.15263724327087402
using linear solver:  direct
epoch:  1	argmax batch num,  3
argmax time taken,  0.12101197242736816
jac:  cuda:0
assembling the matrix time taken:  0.000324249267578125
solving Ax = b time taken:  0.000499725341796875
		 time taken minimize linear layer:  0.0008461475372314453
current error:  tensor(0.1981)
epoch:  2	argmax batch num,  3
argmax time taken,  0.1140599250793457
jac:  cuda:0
assembling the matrix time taken:  0.00037741661071777344
solving Ax = b time taken:  0.0007748603820800781
		 time taken minimize linear layer:  0.0011768341064453125
current error:  tensor(0.1554)
epoch:  3	argmax batch num,  3
argmax time taken,  0.10808086395263672
jac:  cuda:0
assembling the matrix time taken:  0.000339508056640625
solving Ax = b time taken:  0.0009024143218994141
		 time taken minimize linear layer:  0.0012664794921875
current error:  tensor(0.1255)
epoch:  4	argmax batch num,  3
argmax time taken,  0.10809588432312012
jac:  cuda:0
as

argmax time taken,  0.10909342765808105
jac:  cuda:0
assembling the matrix time taken:  0.00036215782165527344
solving Ax = b time taken:  0.0027205944061279297
		 time taken minimize linear layer:  0.003106832504272461
current error:  tensor(0.0543)
epoch:  31	argmax batch num,  3
argmax time taken,  0.10923147201538086
jac:  cuda:0
assembling the matrix time taken:  0.00033020973205566406
solving Ax = b time taken:  0.0027937889099121094
		 time taken minimize linear layer:  0.003148794174194336
current error:  tensor(0.0532)
epoch:  32	argmax batch num,  3
argmax time taken,  0.10926938056945801
jac:  cuda:0
assembling the matrix time taken:  0.00035452842712402344
solving Ax = b time taken:  0.002830982208251953
		 time taken minimize linear layer:  0.0032105445861816406
current error:  tensor(0.0515)
epoch:  33	argmax batch num,  3
argmax time taken,  0.10910272598266602
jac:  cuda:0
assembling the matrix time taken:  0.0003418922424316406
solving Ax = b time taken:  0.00313591957

argmax time taken,  0.11348867416381836
jac:  cuda:0
assembling the matrix time taken:  0.0003688335418701172
solving Ax = b time taken:  0.004833698272705078
		 time taken minimize linear layer:  0.005227088928222656
current error:  tensor(0.0238)
epoch:  61	argmax batch num,  3
argmax time taken,  0.11365866661071777
jac:  cuda:0
assembling the matrix time taken:  0.00033211708068847656
solving Ax = b time taken:  0.004958629608154297
		 time taken minimize linear layer:  0.005315065383911133
current error:  tensor(0.0235)
epoch:  62	argmax batch num,  3
argmax time taken,  0.11356616020202637
jac:  cuda:0
assembling the matrix time taken:  0.0003628730773925781
solving Ax = b time taken:  0.005021810531616211
		 time taken minimize linear layer:  0.005409955978393555
current error:  tensor(0.0231)
epoch:  63	argmax batch num,  3
argmax time taken,  0.11359691619873047
jac:  cuda:0
assembling the matrix time taken:  0.00033545494079589844
solving Ax = b time taken:  0.005111932754516

argmax time taken,  0.1176154613494873
jac:  cuda:0
assembling the matrix time taken:  0.00036215782165527344
solving Ax = b time taken:  0.008771419525146484
		 time taken minimize linear layer:  0.009158611297607422
current error:  tensor(0.0126)
epoch:  91	argmax batch num,  3
argmax time taken,  0.11468863487243652
jac:  cuda:0
assembling the matrix time taken:  0.0003485679626464844
solving Ax = b time taken:  0.008597850799560547
		 time taken minimize linear layer:  0.008971691131591797
current error:  tensor(0.0124)
epoch:  92	argmax batch num,  3
argmax time taken,  0.11472105979919434
jac:  cuda:0
assembling the matrix time taken:  0.00037097930908203125
solving Ax = b time taken:  0.00846242904663086
		 time taken minimize linear layer:  0.008858442306518555
current error:  tensor(0.0123)
epoch:  93	argmax batch num,  3
argmax time taken,  0.11470222473144531
jac:  cuda:0
assembling the matrix time taken:  0.0003418922424316406
solving Ax = b time taken:  0.00862526893615722

current error:  tensor(0.0097)
epoch:  121	argmax batch num,  3
argmax time taken,  0.11927914619445801
jac:  cuda:0
assembling the matrix time taken:  0.00036525726318359375
solving Ax = b time taken:  0.010375499725341797
		 time taken minimize linear layer:  0.010766029357910156
current error:  tensor(0.0096)
epoch:  122	argmax batch num,  3
argmax time taken,  0.1192464828491211
jac:  cuda:0
assembling the matrix time taken:  0.00037670135498046875
solving Ax = b time taken:  0.010858535766601562
		 time taken minimize linear layer:  0.011259794235229492
current error:  tensor(0.0095)
epoch:  123	argmax batch num,  3
argmax time taken,  0.11935019493103027
jac:  cuda:0
assembling the matrix time taken:  0.00035452842712402344
solving Ax = b time taken:  0.010520696640014648
		 time taken minimize linear layer:  0.010900020599365234
current error:  tensor(0.0094)
epoch:  124	argmax batch num,  3
argmax time taken,  0.11592602729797363
jac:  cuda:0
assembling the matrix time taken:  

...

current error:  tensor(0.0047)
epoch:  390	argmax batch num,  3
argmax time taken,  0.12943100929260254
jac:  cuda:0
assembling the matrix time taken:  0.0005145072937011719
solving Ax = b time taken:  0.0512237548828125
		 time taken minimize linear layer:  0.05176401138305664
current error:  tensor(0.0047)
epoch:  391	argmax batch num,  3
argmax time taken,  0.12900018692016602
jac:  cuda:0
assembling the matrix time taken:  0.007173299789428711
solving Ax = b time taken:  0.04829716682434082
		 time taken minimize linear layer:  0.05562114715576172
current error:  tensor(0.0047)
epoch:  392	argmax batch num,  3
argmax time taken,  0.1294386386871338
jac:  cuda:0
assembling the matrix time taken:  0.0005190372467041016
solving Ax = b time taken:  0.05067253112792969
		 time taken minimize linear layer:  0.051343441009521484
current error:  tensor(0.0047)
epoch:  393	argmax batch num,  3
argmax time taken,  0.12961220741271973
jac:  cuda:0
assembling the matrix time taken:  0.00052094

solving Ax = b time taken:  0.050269365310668945
		 time taken minimize linear layer:  0.0509343147277832
current error:  tensor(0.0043)
epoch:  420	argmax batch num,  3
argmax time taken,  0.1304638385772705
jac:  cuda:0
assembling the matrix time taken:  0.0005092620849609375
solving Ax = b time taken:  0.05027437210083008
		 time taken minimize linear layer:  0.05093550682067871
current error:  tensor(0.0043)
epoch:  421	argmax batch num,  3
argmax time taken,  0.13077521324157715
jac:  cuda:0
assembling the matrix time taken:  0.0005185604095458984
solving Ax = b time taken:  0.05039620399475098
		 time taken minimize linear layer:  0.05106687545776367
current error:  tensor(0.0043)
epoch:  422	argmax batch num,  3
argmax time taken,  0.13104844093322754
jac:  cuda:0
assembling the matrix time taken:  0.0005128383636474609
solving Ax = b time taken:  0.05035758018493652
		 time taken minimize linear layer:  0.05102348327636719
current error:  tensor(0.0043)
epoch:  423	argmax batch

argmax time taken,  0.13149094581604004
jac:  cuda:0
assembling the matrix time taken:  0.0005464553833007812
solving Ax = b time taken:  0.05923962593078613
		 time taken minimize linear layer:  0.059812068939208984
current error:  tensor(0.0039)
epoch:  450	argmax batch num,  3
argmax time taken,  0.13553118705749512
jac:  cuda:0
assembling the matrix time taken:  0.0005712509155273438
solving Ax = b time taken:  0.059411048889160156
		 time taken minimize linear layer:  0.06000852584838867
current error:  tensor(0.0039)
epoch:  451	argmax batch num,  3
argmax time taken,  0.13251662254333496
jac:  cuda:0
assembling the matrix time taken:  0.0005280971527099609
solving Ax = b time taken:  0.059551239013671875
		 time taken minimize linear layer:  0.06010556221008301
current error:  tensor(0.0039)
epoch:  452	argmax batch num,  3
argmax time taken,  0.13209319114685059
jac:  cuda:0
assembling the matrix time taken:  0.0005655288696289062
solving Ax = b time taken:  0.0582275390625
		 

argmax time taken,  0.13923048973083496
jac:  cuda:0
assembling the matrix time taken:  0.0005538463592529297
solving Ax = b time taken:  0.061107635498046875
		 time taken minimize linear layer:  0.06168770790100098
current error:  tensor(0.0036)
epoch:  480	argmax batch num,  3
argmax time taken,  0.13322663307189941
jac:  cuda:0
assembling the matrix time taken:  0.0006430149078369141
solving Ax = b time taken:  0.060837745666503906
		 time taken minimize linear layer:  0.061505794525146484
current error:  tensor(0.0036)
epoch:  481	argmax batch num,  3
argmax time taken,  0.13328266143798828
jac:  cuda:0
assembling the matrix time taken:  0.0005464553833007812
solving Ax = b time taken:  0.06143379211425781
		 time taken minimize linear layer:  0.06200671195983887
current error:  tensor(0.0036)
epoch:  482	argmax batch num,  3
argmax time taken,  0.1332845687866211
jac:  cuda:0
assembling the matrix time taken:  0.0005404949188232422
solving Ax = b time taken:  0.061441659927368164

argmax time taken,  0.1342761516571045
jac:  cuda:0
assembling the matrix time taken:  0.0005753040313720703
solving Ax = b time taken:  0.06414103507995605
		 time taken minimize linear layer:  0.0647425651550293
current error:  tensor(0.0032)
epoch:  510	argmax batch num,  3
argmax time taken,  0.13425087928771973
jac:  cuda:0
assembling the matrix time taken:  0.0005908012390136719
solving Ax = b time taken:  0.06427216529846191
		 time taken minimize linear layer:  0.06489014625549316
current error:  tensor(0.0032)
epoch:  511	argmax batch num,  3
argmax time taken,  0.1345350742340088
jac:  cuda:0
assembling the matrix time taken:  0.0005745887756347656
solving Ax = b time taken:  0.06421613693237305
		 time taken minimize linear layer:  0.06481647491455078
current error:  tensor(0.0032)
epoch:  512	argmax batch num,  3
argmax time taken,  0.13735389709472656
jac:  cuda:0
assembling the matrix time taken:  0.0005967617034912109
solving Ax = b time taken:  0.06382012367248535
		 ti

## other examples... 

In [10]:

def target(x): # product of sin(pi x_i) 
#         return torch.prod(torch.sin(pi * x), dim=1,keepdim = True)
    return torch.sum(torch.sin(pi * x), dim=1,keepdim = True)

dim = 10 
function_name = "10Dsine-sum"
filename_write = "data/1DOGA-{}-order.txt".format(function_name)
M = 2**20 # around2**19 50w 
f_write = open(filename_write, "a")
f_write.write("Integration points: Quasi Monte Carlo:  {}\n".format(M))
f_write.close() 
save = True 
write2file = True
memory = 2**30 
for relu_k in [4]: 
    s = 1 
    for N0 in [2**11]: 
        print()
        print() 
        N = s*N0 
        exponent = 9    
        num_epochs=  2**exponent 
        my_model = None 

        err, my_model = OGAL2FittingReLU4Dplus_QMC(my_model,target, \
                    s,N0,num_epochs, M, k = relu_k, linear_solver = "direct", memory=memory)

        if save: 
            folder = 'data/'
            filename = folder + function_name + "_err_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(err,filename)
            filename = folder + function_name +  "_model_randDict_relu_{}_size_{}_num_neurons_{}.pt".format(relu_k,s * N0,num_epochs)
            torch.save(my_model.state_dict(),filename) 

        show_convergence_order(err,exponent,N,filename_write,write2file = write2file)
        show_convergence_order_latex(err,exponent,k=relu_k,d=dim)





generate sob sequence: 0.127044677734375
using linear solver:  direct
epoch:  1	argmax batch num,  3
argmax time taken,  0.1189718246459961
jac:  cuda:0
assembling the matrix time taken:  0.0002906322479248047
solving Ax = b time taken:  0.0003325939178466797
		 time taken minimize linear layer:  0.0006453990936279297
current error:  tensor(1.9962)
epoch:  2	argmax batch num,  3
argmax time taken,  0.10778617858886719
jac:  cuda:0
assembling the matrix time taken:  0.0003409385681152344
solving Ax = b time taken:  0.0006866455078125
		 time taken minimize linear layer:  0.0010509490966796875
current error:  tensor(1.7205)
epoch:  3	argmax batch num,  3
argmax time taken,  0.10795474052429199
jac:  cuda:0
assembling the matrix time taken:  0.00031876564025878906
solving Ax = b time taken:  0.0007979869842529297
		 time taken minimize linear layer:  0.0011413097381591797
current error:  tensor(1.3291)
epoch:  4	argmax batch num,  3
argmax time taken,  0.10811376571655273
jac:  cuda:0
a

argmax time taken,  0.10913491249084473
jac:  cuda:0
assembling the matrix time taken:  0.00029754638671875
solving Ax = b time taken:  0.002677440643310547
		 time taken minimize linear layer:  0.0029993057250976562
current error:  tensor(0.6481)
epoch:  32	argmax batch num,  3
argmax time taken,  0.10920310020446777
jac:  cuda:0
assembling the matrix time taken:  0.0003170967102050781
solving Ax = b time taken:  0.0027968883514404297
		 time taken minimize linear layer:  0.0031385421752929688
current error:  tensor(0.6290)
epoch:  33	argmax batch num,  3
argmax time taken,  0.10916018486022949
jac:  cuda:0
assembling the matrix time taken:  0.000301361083984375
solving Ax = b time taken:  0.0030226707458496094
		 time taken minimize linear layer:  0.0033485889434814453
current error:  tensor(0.6155)
epoch:  34	argmax batch num,  3
argmax time taken,  0.11244654655456543
jac:  cuda:0
assembling the matrix time taken:  0.0003197193145751953
solving Ax = b time taken:  0.002969741821289

argmax time taken,  0.11350631713867188
jac:  cuda:0
assembling the matrix time taken:  0.00030112266540527344
solving Ax = b time taken:  0.0048372745513916016
		 time taken minimize linear layer:  0.005164146423339844
current error:  tensor(0.2743)
epoch:  62	argmax batch num,  3
argmax time taken,  0.11352777481079102
jac:  cuda:0
assembling the matrix time taken:  0.0003306865692138672
solving Ax = b time taken:  0.004878997802734375
		 time taken minimize linear layer:  0.005234718322753906
current error:  tensor(0.2618)
epoch:  63	argmax batch num,  3
argmax time taken,  0.11360335350036621
jac:  cuda:0
assembling the matrix time taken:  0.00030231475830078125
solving Ax = b time taken:  0.0049800872802734375
		 time taken minimize linear layer:  0.005307197570800781
current error:  tensor(0.2517)
epoch:  64	argmax batch num,  3
argmax time taken,  0.11374163627624512
jac:  cuda:0
assembling the matrix time taken:  0.0003299713134765625
solving Ax = b time taken:  0.0049641132354

argmax time taken,  0.11470627784729004
jac:  cuda:0
assembling the matrix time taken:  0.00030803680419921875
solving Ax = b time taken:  0.008445262908935547
		 time taken minimize linear layer:  0.008778095245361328
current error:  tensor(0.1389)
epoch:  92	argmax batch num,  3
argmax time taken,  0.1147162914276123
jac:  cuda:0
assembling the matrix time taken:  0.00032782554626464844
solving Ax = b time taken:  0.008359670639038086
		 time taken minimize linear layer:  0.008711576461791992
current error:  tensor(0.1378)
epoch:  93	argmax batch num,  3
argmax time taken,  0.11476445198059082
jac:  cuda:0
assembling the matrix time taken:  0.0003120899200439453
solving Ax = b time taken:  0.00850534439086914
		 time taken minimize linear layer:  0.008841514587402344
current error:  tensor(0.1359)
epoch:  94	argmax batch num,  3
argmax time taken,  0.1148221492767334
jac:  cuda:0
assembling the matrix time taken:  0.00032639503479003906
solving Ax = b time taken:  0.00879907608032226

epoch:  122	argmax batch num,  3
argmax time taken,  0.11926102638244629
jac:  cuda:0
assembling the matrix time taken:  0.00034928321838378906
solving Ax = b time taken:  0.01067209243774414
		 time taken minimize linear layer:  0.01104593276977539
current error:  tensor(0.0932)
epoch:  123	argmax batch num,  3
argmax time taken,  0.11939239501953125
jac:  cuda:0
assembling the matrix time taken:  0.0003211498260498047
solving Ax = b time taken:  0.010344743728637695
		 time taken minimize linear layer:  0.010690450668334961
current error:  tensor(0.0917)
epoch:  124	argmax batch num,  3
argmax time taken,  0.11591863632202148
jac:  cuda:0
assembling the matrix time taken:  0.00034999847412109375
solving Ax = b time taken:  0.010409116744995117
		 time taken minimize linear layer:  0.01078343391418457
current error:  tensor(0.0909)
epoch:  125	argmax batch num,  3
argmax time taken,  0.11945700645446777
jac:  cuda:0
assembling the matrix time taken:  0.0003192424774169922
solving Ax =

...

argmax time taken,  0.1296236515045166
jac:  cuda:0
assembling the matrix time taken:  0.00048279762268066406
solving Ax = b time taken:  0.04788994789123535
		 time taken minimize linear layer:  0.04839825630187988
current error:  tensor(0.0434)
epoch:  390	argmax batch num,  3
argmax time taken,  0.1293048858642578
jac:  cuda:0
assembling the matrix time taken:  0.0004858970642089844
solving Ax = b time taken:  0.05113506317138672
		 time taken minimize linear layer:  0.051645755767822266
current error:  tensor(0.0434)
epoch:  391	argmax batch num,  3
argmax time taken,  0.12987232208251953
jac:  cuda:0
assembling the matrix time taken:  0.000476837158203125
solving Ax = b time taken:  0.04813671112060547
		 time taken minimize linear layer:  0.048639774322509766
current error:  tensor(0.0433)
epoch:  392	argmax batch num,  3
argmax time taken,  0.12956976890563965
jac:  cuda:0
assembling the matrix time taken:  0.00048160552978515625
solving Ax = b time taken:  0.05049014091491699
	

argmax time taken,  0.1307220458984375
jac:  cuda:0
assembling the matrix time taken:  0.00048232078552246094
solving Ax = b time taken:  0.049927711486816406
		 time taken minimize linear layer:  0.050435781478881836
current error:  tensor(0.0414)
epoch:  420	argmax batch num,  3
argmax time taken,  0.13043832778930664
jac:  cuda:0
assembling the matrix time taken:  0.00049591064453125
solving Ax = b time taken:  0.04998588562011719
		 time taken minimize linear layer:  0.050507545471191406
current error:  tensor(0.0413)
epoch:  421	argmax batch num,  3
argmax time taken,  0.13060402870178223
jac:  cuda:0
assembling the matrix time taken:  0.0004949569702148438
solving Ax = b time taken:  0.049985408782958984
		 time taken minimize linear layer:  0.050507545471191406
current error:  tensor(0.0413)
epoch:  422	argmax batch num,  3
argmax time taken,  0.1308307647705078
jac:  cuda:0
assembling the matrix time taken:  0.0004956722259521484
solving Ax = b time taken:  0.04996490478515625


argmax time taken,  0.1315615177154541
jac:  cuda:0
assembling the matrix time taken:  0.0005016326904296875
solving Ax = b time taken:  0.059163808822631836
		 time taken minimize linear layer:  0.0596926212310791
current error:  tensor(0.0392)
epoch:  450	argmax batch num,  3
argmax time taken,  0.13515806198120117
jac:  cuda:0
assembling the matrix time taken:  0.0005245208740234375
solving Ax = b time taken:  0.059650421142578125
		 time taken minimize linear layer:  0.060201406478881836
current error:  tensor(0.0391)
epoch:  451	argmax batch num,  3
argmax time taken,  0.13244080543518066
jac:  cuda:0
assembling the matrix time taken:  0.0005095005035400391
solving Ax = b time taken:  0.05904412269592285
		 time taken minimize linear layer:  0.05957913398742676
current error:  tensor(0.0390)
epoch:  452	argmax batch num,  3
argmax time taken,  0.13207554817199707
jac:  cuda:0
assembling the matrix time taken:  0.0005354881286621094
solving Ax = b time taken:  0.058333635330200195


argmax time taken,  0.13956379890441895
jac:  cuda:0
assembling the matrix time taken:  0.0005147457122802734
solving Ax = b time taken:  0.06072378158569336
		 time taken minimize linear layer:  0.0612640380859375
current error:  tensor(0.0370)
epoch:  480	argmax batch num,  3
argmax time taken,  0.13327312469482422
jac:  cuda:0
assembling the matrix time taken:  0.0005266666412353516
solving Ax = b time taken:  0.06084442138671875
		 time taken minimize linear layer:  0.06140017509460449
current error:  tensor(0.0369)
epoch:  481	argmax batch num,  3
argmax time taken,  0.13927960395812988
jac:  cuda:0
assembling the matrix time taken:  0.000507354736328125
solving Ax = b time taken:  0.06115984916687012
		 time taken minimize linear layer:  0.06169247627258301
current error:  tensor(0.0369)
epoch:  482	argmax batch num,  3
argmax time taken,  0.13334107398986816
jac:  cuda:0
assembling the matrix time taken:  0.0005052089691162109
solving Ax = b time taken:  0.06102585792541504
		 t

argmax time taken,  0.1374666690826416
jac:  cuda:0
assembling the matrix time taken:  0.0005409717559814453
solving Ax = b time taken:  0.06393218040466309
		 time taken minimize linear layer:  0.0644996166229248
current error:  tensor(0.0354)
epoch:  510	argmax batch num,  3
argmax time taken,  0.13427495956420898
jac:  cuda:0
assembling the matrix time taken:  0.0005407333374023438
solving Ax = b time taken:  0.06396055221557617
		 time taken minimize linear layer:  0.0645284652709961
current error:  tensor(0.0353)
epoch:  511	argmax batch num,  3
argmax time taken,  0.13761138916015625
jac:  cuda:0
assembling the matrix time taken:  0.0005409717559814453
solving Ax = b time taken:  0.06391596794128418
		 time taken minimize linear layer:  0.0644826889038086
current error:  tensor(0.0353)
epoch:  512	argmax batch num,  3
argmax time taken,  0.13431882858276367
jac:  cuda:0
assembling the matrix time taken:  0.0005159378051757812
solving Ax = b time taken:  0.0636444091796875
		 time